In [1]:
import os
from pydub import AudioSegment
genres = 0
genre_names = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

os.mkdir('data_wav')
#cur_time = time.time()
while genres<10:
    os.mkdir('data_wav/' + genre_names[genres])
    folder_name = genre_names[genres]
    songs = os.listdir('music-genre-classification/data/' + folder_name)
    print('processing folder: ', folder_name)
    song_number = 0
    while song_number<10:
        filename = songs[song_number]
        path = 'music-genre-classification/data/'+folder_name+'/'+filename
        song = AudioSegment.from_file(path,'au')
        path_export = 'data_wav/'+folder_name+'/'+str(song_number)+'.wav'
        song.export(path_export,format='wav')
        song_number += 1
    genres +=1
#final_time = time.time()
#time_taken = final_time - cur_time
print('Au files converted!')

FileExistsError: [Errno 17] File exists: 'data_wav'

In [10]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
from numpy.lib import stride_tricks

""" short time fourier transform of audio signal """
def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))

    # zeros at beginning (thus center of 1st window should be for sample nr. 0)   
    samples = np.append(np.zeros(int(np.floor(frameSize/2.0))), sig)    
    # cols for windowing
    cols = np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))

    frames = stride_tricks.as_strided(samples, shape=(int(cols), frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win

    return np.fft.rfft(frames)    

""" scale frequency axis logarithmically """    
def logscale_spec(spec, sr=44100, factor=20.):
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))

    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):        
        if i == len(scale)-1:
            newspec[:,i] = np.sum(spec[:,int(scale[i]):], axis=1)
        else:        
            newspec[:,i] = np.sum(spec[:,int(scale[i]):int(scale[i+1])], axis=1)

    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[int(scale[i]):])]
        else:
            freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]

    return newspec, freqs

""" plot spectrogram"""
def plotstft(audiopath, plotpath, binsize=2**10, colormap="jet"):
    samplerate, samples = wav.read(audiopath)

    s = stft(samples, binsize)

    sshow, freq = logscale_spec(s, factor=1.0, sr=samplerate)

    ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel

    timebins, freqbins = np.shape(ims)

    print("timebins: ", timebins)
    print("freqbins: ", freqbins)

    plt.figure(figsize=(15, 7.5))
    plt.imshow(np.transpose(ims), origin="lower", aspect="auto", cmap=colormap, interpolation="none")
    plt.colorbar()

    plt.xlabel("time (s)")
    plt.ylabel("frequency (hz)")
    plt.xlim([0, timebins-1])
    plt.ylim([0, freqbins])

    xlocs = np.float32(np.linspace(0, timebins-1, 5))
    plt.xticks(xlocs, ["%.02f" % l for l in ((xlocs*len(samples)/timebins)+(0.5*binsize))/samplerate])
    ylocs = np.int16(np.round(np.linspace(0, freqbins-1, 10)))
    plt.yticks(ylocs, ["%.02f" % freq[i] for i in ylocs])

    if plotpath:
        plt.savefig(plotpath, bbox_inches="tight")
    else:
        plt.show()

    plt.clf()

    return ims

#ims = plotstft(filepath)

In [16]:
genres_test = 0
song_number_test = 0
genre_names_test = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

os.mkdir('images')
#cur_time = time.time()
while genres_test<10:
    os.mkdir('images/' + genre_names[genres])
    song_number_test = 0
    while song_number_test < 10:
        img = plotstft(audiopath = 'data_wav/'+ genre_names_test[genres_test] + '/' + str(song_number_test) + '.wav', plotpath = 'images/'+ genre_names_test[genres_test] + '/' + str(song_number_test) + '.png')
        song_number_test += 1

    print(genre_names_test[genres_test] + ' proccesed!')
    genres += 1

timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
blues proccesed!
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
blues proccesed!
timebins:  1293
freqbins:  513


/Users/zixel/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
blues proccesed!
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
blues proccesed!
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
timebins:  1293
freqbins:  513
blues proccesed!
timebins:  1293
freqbins:  513
timebins:  1293
fre

IndexError: list index out of range

<Figure size 1080x540 with 0 Axes>

...

<Figure size 1080x540 with 0 Axes>

<Figure size 1080x540 with 0 Axes>

<Figure size 1080x540 with 0 Axes>

<Figure size 1080x540 with 0 Axes>

<Figure size 1080x540 with 0 Axes>